In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_data(imageDir, imgSize=(150, 150)):
    images = []
    labels = []
    
    for label in ["NORMAL", "PNEUMONIA"]:
        folderPath= os.path.join(imageDir,label)
        for filename in os.listdir(folderPath):
            if filename.endswith(".jpeg"):
                imgPath = os.path.join(folderPath,filename)
                img = cv2.imread(imgPath)
                img = cv2.resize(img, imgSize)
                images.append(img)
                labels.append(0 if label == "NORMAL" else 1)
    images=np.array(images)
    labels=np.array(labels)
    return images, labels

imageDir="./chest_xray"
images, labels = load_data(imageDir)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential(
    [
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ]
)

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()



In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
validation_generator = test_datagen.flow(X_test, y_test, batch_size=32)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 32,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(X_test) // 32
)


In [ ]:
loss, accuracy = model.evaluate(validation_generator, steps=len(X_test) // 32)
print(f"Test accuracy: {accuracy*100:.2f}%")


In [ ]:
# Load a new image for prediction
img_path = 'path_to_new_image.jpg'
img = cv2.imread(img_path)
img = cv2.resize(img, (150, 150)) 
img = np.expand_dims(img, axis=0) 

# Predict
prediction = model.predict(img)
if prediction[0] > 0.5:
    print("Prediction: Pneumonia")
else:
    print("Prediction: Normal")


In [ ]:
model.save("pneumonia_detection_model.h5")
